In [1]:
import pandas as pd

In [2]:
customer = pd.read_csv('./customer_join.csv')

In [3]:
customer.shape

(4192, 18)

In [4]:
customer.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4192 entries, 0 to 4191
Data columns (total 18 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   customer_id        4192 non-null   object 
 1   name               4192 non-null   object 
 2   class              4192 non-null   object 
 3   gender             4192 non-null   object 
 4   start_date         4192 non-null   object 
 5   end_date           1350 non-null   object 
 6   campaign_id        4192 non-null   object 
 7   is_deleted         4192 non-null   int64  
 8   class_name         4192 non-null   object 
 9   price              4192 non-null   int64  
 10  campaign_name      4192 non-null   object 
 11  mean               4192 non-null   float64
 12  median             4192 non-null   float64
 13  max                4192 non-null   int64  
 14  min                4192 non-null   int64  
 15  routine_flg        4192 non-null   int64  
 16  calc_date          4192 

In [5]:
customer.head(2)

,customer_id,name,class,gender,start_date,end_date,campaign_id,is_deleted,class_name,price,campaign_name,mean,median,max,min,routine_flg,calc_date,membership_period
0,OA832399,XXXX,C01,F,2015-05-01,NaN,CA1,0,종일,10500,일반,4.833333,5.0,8,2,1,2019-04-30,47
1,PL270116,XXXXX,C01,M,2015-05-01,NaN,CA1,0,종일,10500,일반,5.083333,5.0,7,3,1,2019-04-30,47


In [6]:
uselog_months = pd.read_csv('./use_log_months.csv')

In [7]:
uselog_months.shape

(36842, 3)

In [8]:
uselog_months.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 36842 entries, 0 to 36841
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   연월           36842 non-null  int64 
 1   customer_id  36842 non-null  object
 2   count        36842 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 863.6+ KB


In [9]:
uselog_months.head(2)

,연월,customer_id,count
0,201804,AS002855,4
1,201804,AS009013,2


In [10]:
year_months = list(uselog_months['연월'].unique())
year_months

[201804,
 201805,
 201806,
 201807,
 201808,
 201809,
 201810,
 201811,
 201812,
 201901,
 201902,
 201903]

In [11]:
uselog = pd.DataFrame()
for i in range(len(year_months)):
    # 현재 달
    tmp = uselog_months.loc[uselog_months['연월']==year_months[i]]
    tmp.rename(columns={'count':'count_0'}, inplace=True)
    # 1달 전
    tmp_before = uselog_months.loc[uselog_months['연월']==year_months[i-1]]
    tmp_before.rename(columns={'count':'count_1'}, inplace=True)
    # 필요없는 column 제거                               
    del tmp_before['연월']
    # 연결
    tmp = pd.merge(tmp, tmp_before, on='customer_id', how='left')
    uselog = pd.concat([uselog, tmp], ignore_index=True)

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\frame.py:4133: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [12]:
tmp.head(2)

,연월,customer_id,count_0,count_1
0,201903,AS002855,4,5.0
1,201903,AS008805,1,6.0


In [13]:
tmp.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2953 entries, 0 to 2952
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   연월           2953 non-null   int64  
 1   customer_id  2953 non-null   object 
 2   count_0      2953 non-null   int64  
 3   count_1      2888 non-null   float64
dtypes: float64(1), int64(2), object(1)
memory usage: 115.4+ KB


In [14]:
uselog.head(2)

,연월,customer_id,count_0,count_1
0,201804,AS002855,4,4.0
1,201804,AS009013,2,NaN


In [15]:
uselog.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 36842 entries, 0 to 36841
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   연월           36842 non-null  int64  
 1   customer_id  36842 non-null  object 
 2   count_0      36842 non-null  int64  
 3   count_1      34831 non-null  float64
dtypes: float64(1), int64(2), object(1)
memory usage: 1.1+ MB


In [16]:
from dateutil.relativedelta import relativedelta

In [17]:
# 탈퇴한 사람끼리, 탈퇴 안 한 사람끼리 따로 처리
exit_customer = customer.loc[customer['is_deleted']==1]

In [18]:
exit_customer['exit_date'] = None

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [19]:
exit_customer['exit_date'] = pd.to_datetime(exit_customer['end_date'])
exit_customer['exit_date']

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


708    2018-04-30
729    2018-04-30
730    2018-04-30
770    2018-05-31
785    2018-05-31
          ...    
4049   2019-02-28
4050   2019-03-31
4068   2019-03-31
4086   2019-03-31
4099   2019-03-31
Name: exit_date, Length: 1350, dtype: datetime64[ns]

In [20]:
for i in range(len(exit_customer)):
    exit_customer['exit_date'].iloc[i] = exit_customer['exit_date'].iloc[i] - relativedelta(months=1)
exit_customer['exit_date']

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


708    2018-03-30
729    2018-03-30
730    2018-03-30
770    2018-04-30
785    2018-04-30
          ...    
4049   2019-01-28
4050   2019-02-28
4068   2019-02-28
4086   2019-02-28
4099   2019-02-28
Name: exit_date, Length: 1350, dtype: datetime64[ns]

In [21]:
exit_customer['연월'] = exit_customer['exit_date'].dt.strftime('%Y%m')

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [22]:
exit_customer.head(2)

,customer_id,name,class,gender,start_date,end_date,campaign_id,is_deleted,class_name,price,campaign_name,mean,median,max,min,routine_flg,calc_date,membership_period,exit_date,연월
708,TS511179,XXXXXX,C01,F,2016-05-01,2018-04-30,CA1,1,종일,10500,일반,3.0,3.0,3,3,0,2018-04-30,23,2018-03-30,201803
729,TS443736,XXXX,C02,M,2016-05-01,2018-04-30,CA1,1,주간,7500,일반,3.0,3.0,3,3,0,2018-04-30,23,2018-03-30,201803


In [23]:
uselog['연월'] = uselog['연월'].astype(str)

In [24]:
exit_uselog = pd.merge(uselog, exit_customer, on=['customer_id', '연월'], how='left')

In [25]:
exit_uselog.shape

(36842, 22)

In [26]:
exit_uselog.sample(5)

,연월,customer_id,count_0,count_1,name,class,gender,start_date,end_date,campaign_id,...,price,campaign_name,mean,median,max,min,routine_flg,calc_date,membership_period,exit_date
14657,201808,PL087063,5,6.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT
28898,201901,HI086872,7,5.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT
36367,201903,PL819882,6,3.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT
29745,201901,OA251022,8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT
12275,201807,TS964483,11,9.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT


In [27]:
# name이 비어있는 행 제거
exit_uselog = exit_uselog.dropna(subset=['name'])

In [28]:
exit_uselog.sample(5)

,연월,customer_id,count_0,count_1,name,class,gender,start_date,end_date,campaign_id,...,price,campaign_name,mean,median,max,min,routine_flg,calc_date,membership_period,exit_date
4884,201805,OA003684,5,4.0,XXXXX,C02,M,2017-09-01,2018-06-30,CA1,...,7500.0,일반,4.000000,4.0,5.0,3.0,0.0,2018-06-30,9.0,2018-05-30
30755,201901,TS794667,1,3.0,XX,C01,M,2017-11-01,2019-02-28,CA1,...,10500.0,일반,4.181818,4.0,7.0,1.0,1.0,2019-02-28,15.0,2019-01-28
4317,201805,HI554088,1,4.0,XX,C03,M,2017-07-01,2018-06-30,CA2,...,6000.0,입회비반액할인,2.000000,1.0,4.0,1.0,0.0,2018-06-30,11.0,2018-05-30
8496,201806,PL312029,3,4.0,XXXX,C02,F,2018-02-01,2018-07-31,CA1,...,7500.0,일반,3.250000,3.0,4.0,3.0,0.0,2018-07-31,5.0,2018-06-30
11963,201807,TS169261,2,1.0,XXXXX,C03,M,2017-12-01,2018-08-31,CA3,...,6000.0,입회비무료,2.600000,2.0,5.0,1.0,0.0,2018-08-31,8.0,2018-07-31


In [29]:
exit_uselog.isnull().sum()

연월                     0
customer_id            0
count_0                0
count_1              177
name                   0
class                  0
gender                 0
start_date             0
end_date               0
campaign_id            0
is_deleted             0
class_name             0
price                  0
campaign_name          0
mean                   0
median                 0
max                    0
min                    0
routine_flg            0
calc_date              0
membership_period      0
exit_date              0
dtype: int64

In [30]:
conti_customer = customer.loc[customer['is_deleted']==0]

In [31]:
conti_uselog = pd.merge(uselog, conti_customer, on=['customer_id'], how='left')

In [32]:
conti_uselog.shape

(36842, 21)

In [33]:
conti_uselog.sample(5)

,연월,customer_id,count_0,count_1,name,class,gender,start_date,end_date,campaign_id,...,class_name,price,campaign_name,mean,median,max,min,routine_flg,calc_date,membership_period
33930,201903,AS118198,7,7.0,XX,C03,M,2018-12-09,NaN,CA3,...,야간,6000.0,입회비무료,8.750000,8.5,11.0,7.0,1.0,2019-04-30,4.0
23230,201811,IK339856,7,3.0,XXXX,C02,M,2015-11-01,NaN,CA1,...,주간,7500.0,일반,4.333333,4.0,7.0,2.0,1.0,2019-04-30,41.0
9191,201807,AS082596,6,8.0,XXXXX,C01,M,2018-01-01,NaN,CA1,...,종일,10500.0,일반,6.500000,6.5,9.0,4.0,1.0,2019-04-30,15.0
2971,201804,TS953790,11,6.0,XXXXX,C02,F,2018-01-01,NaN,CA1,...,주간,7500.0,일반,8.166667,8.0,11.0,6.0,1.0,2019-04-30,15.0
8692,201806,PL827485,3,4.0,XXX,C01,F,2015-07-01,NaN,CA1,...,종일,10500.0,일반,4.416667,4.0,8.0,1.0,1.0,2019-04-30,45.0


In [34]:
conti_uselog.isnull().sum()

연월                       0
customer_id              0
count_0                  0
count_1               2011
name                  7266
class                 7266
gender                7266
start_date            7266
end_date             36842
campaign_id           7266
is_deleted            7266
class_name            7266
price                 7266
campaign_name         7266
mean                  7266
median                7266
max                   7266
min                   7266
routine_flg           7266
calc_date             7266
membership_period     7266
dtype: int64

In [35]:
# name이 비어있는 행 제거
conti_uselog = conti_uselog.dropna(subset=['name'])

In [36]:
# 전체를 섞겠다, 인덱스 리셋
conti_uselog = conti_uselog.sample(frac=1).reset_index(drop=True)

In [37]:
conti_uselog

,연월,customer_id,count_0,count_1,name,class,gender,start_date,end_date,campaign_id,...,class_name,price,campaign_name,mean,median,max,min,routine_flg,calc_date,membership_period
0,201809,IK174853,3,4.0,XXXXXX,C01,F,2017-01-01,NaN,CA1,...,종일,10500.0,일반,4.166667,4.0,7.0,3.0,1.0,2019-04-30,27.0
1,201806,HD014166,4,2.0,XXXXX,C02,F,2015-12-01,NaN,CA1,...,주간,7500.0,일반,4.666667,4.5,8.0,2.0,1.0,2019-04-30,40.0
2,201805,OA316605,7,8.0,XXXX,C01,M,2017-11-01,NaN,CA1,...,종일,10500.0,일반,6.250000,6.0,9.0,4.0,1.0,2019-04-30,17.0
3,201810,PL407755,4,6.0,XXXXXX,C03,M,2016-04-01,NaN,CA1,...,야간,6000.0,일반,3.916667,4.0,6.0,1.0,1.0,2019-04-30,36.0
4,201806,TS803730,5,5.0,XXXXX,C03,M,2015-08-01,NaN,CA1,...,야간,6000.0,일반,4.750000,5.0,6.0,3.0,1.0,2019-04-30,44.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29571,201805,OA832624,4,8.0,XXXXX,C03,M,2016-11-01,NaN,CA1,...,야간,6000.0,일반,5.583333,5.0,9.0,4.0,1.0,2019-04-30,29.0
29572,201805,GD639016,6,3.0,XXXX,C01,M,2016-05-01,NaN,CA1,...,종일,10500.0,일반,4.666667,5.0,6.0,3.0,1.0,2019-04-30,35.0
29573,201812,OA575737,8,8.0,XXXXX,C01,F,2017-08-01,NaN,CA2,...,종일,10500.0,입회비반액할인,6.333333,6.0,9.0,4.0,1.0,2019-04-30,20.0
29574,201810,GD323923,6,6.0,XX,C01,M,2015-10-01,NaN,CA1,...,종일,10500.0,일반,4.583333,5.0,7.0,2.0,1.0,2019-04-30,42.0


In [38]:
# 같은 customer_id 행 - 하나 남기고 다 제거
conti_uselog = conti_uselog.drop_duplicates(subset=['customer_id'])

In [39]:
conti_uselog.shape

(2842, 21)

In [40]:
# 계속 하는 사람, 탈퇴한 사람 합치기
predict_data = pd.concat([conti_uselog, exit_uselog], ignore_index=True)

In [41]:
predict_data

,연월,customer_id,count_0,count_1,name,class,gender,start_date,end_date,campaign_id,...,price,campaign_name,mean,median,max,min,routine_flg,calc_date,membership_period,exit_date
0,201809,IK174853,3,4.0,XXXXXX,C01,F,2017-01-01,NaN,CA1,...,10500.0,일반,4.166667,4.0,7.0,3.0,1.0,2019-04-30,27.0,NaT
1,201806,HD014166,4,2.0,XXXXX,C02,F,2015-12-01,NaN,CA1,...,7500.0,일반,4.666667,4.5,8.0,2.0,1.0,2019-04-30,40.0,NaT
2,201805,OA316605,7,8.0,XXXX,C01,M,2017-11-01,NaN,CA1,...,10500.0,일반,6.250000,6.0,9.0,4.0,1.0,2019-04-30,17.0,NaT
3,201810,PL407755,4,6.0,XXXXXX,C03,M,2016-04-01,NaN,CA1,...,6000.0,일반,3.916667,4.0,6.0,1.0,1.0,2019-04-30,36.0,NaT
4,201806,TS803730,5,5.0,XXXXX,C03,M,2015-08-01,NaN,CA1,...,6000.0,일반,4.750000,5.0,6.0,3.0,1.0,2019-04-30,44.0,NaT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4066,201902,TS645212,4,2.0,XXXX,C03,F,2018-03-01,2019-03-31,CA1,...,6000.0,일반,4.500000,4.5,7.0,1.0,0.0,2019-03-31,12.0,2019-02-28
4067,201902,TS741703,5,6.0,XXXX,C03,M,2018-12-08,2019-03-31,CA3,...,6000.0,입회비무료,6.250000,6.0,8.0,5.0,0.0,2019-03-31,3.0,2019-02-28
4068,201902,TS859258,1,3.0,XXXXX,C02,F,2018-12-07,2019-03-31,CA3,...,7500.0,입회비무료,2.500000,2.0,5.0,1.0,0.0,2019-03-31,3.0,2019-02-28
4069,201902,TS886985,5,3.0,XXX,C02,F,2018-03-01,2019-03-31,CA1,...,7500.0,일반,4.250000,4.0,7.0,2.0,1.0,2019-03-31,12.0,2019-02-28


In [42]:
predict_data['period'] = 0

In [43]:
predict_data['now_date'] = pd.to_datetime(predict_data['연월'], format='%Y%m')
predict_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4071 entries, 0 to 4070
Data columns (total 24 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   연월                 4071 non-null   object        
 1   customer_id        4071 non-null   object        
 2   count_0            4071 non-null   int64         
 3   count_1            3679 non-null   float64       
 4   name               4071 non-null   object        
 5   class              4071 non-null   object        
 6   gender             4071 non-null   object        
 7   start_date         4071 non-null   object        
 8   end_date           1229 non-null   object        
 9   campaign_id        4071 non-null   object        
 10  is_deleted         4071 non-null   float64       
 11  class_name         4071 non-null   object        
 12  price              4071 non-null   float64       
 13  campaign_name      4071 non-null   object        
 14  mean    

In [44]:
predict_data['start_date'] = pd.to_datetime(predict_data['start_date'])
predict_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4071 entries, 0 to 4070
Data columns (total 24 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   연월                 4071 non-null   object        
 1   customer_id        4071 non-null   object        
 2   count_0            4071 non-null   int64         
 3   count_1            3679 non-null   float64       
 4   name               4071 non-null   object        
 5   class              4071 non-null   object        
 6   gender             4071 non-null   object        
 7   start_date         4071 non-null   datetime64[ns]
 8   end_date           1229 non-null   object        
 9   campaign_id        4071 non-null   object        
 10  is_deleted         4071 non-null   float64       
 11  class_name         4071 non-null   object        
 12  price              4071 non-null   float64       
 13  campaign_name      4071 non-null   object        
 14  mean    

In [45]:
for i in range(len(predict_data)):
    delta = relativedelta(predict_data['now_date'][i], predict_data['start_date'][i])
    predict_data['period'][i] = delta.years*12+delta.months

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [46]:
predict_data.head()

,연월,customer_id,count_0,count_1,name,class,gender,start_date,end_date,campaign_id,...,mean,median,max,min,routine_flg,calc_date,membership_period,exit_date,period,now_date
0,201809,IK174853,3,4.0,XXXXXX,C01,F,2017-01-01,NaN,CA1,...,4.166667,4.0,7.0,3.0,1.0,2019-04-30,27.0,NaT,20,2018-09-01
1,201806,HD014166,4,2.0,XXXXX,C02,F,2015-12-01,NaN,CA1,...,4.666667,4.5,8.0,2.0,1.0,2019-04-30,40.0,NaT,30,2018-06-01
2,201805,OA316605,7,8.0,XXXX,C01,M,2017-11-01,NaN,CA1,...,6.250000,6.0,9.0,4.0,1.0,2019-04-30,17.0,NaT,6,2018-05-01
3,201810,PL407755,4,6.0,XXXXXX,C03,M,2016-04-01,NaN,CA1,...,3.916667,4.0,6.0,1.0,1.0,2019-04-30,36.0,NaT,30,2018-10-01
4,201806,TS803730,5,5.0,XXXXX,C03,M,2015-08-01,NaN,CA1,...,4.750000,5.0,6.0,3.0,1.0,2019-04-30,44.0,NaT,34,2018-06-01


In [47]:
predict_data.isnull().sum()

연월                      0
customer_id             0
count_0                 0
count_1               392
name                    0
class                   0
gender                  0
start_date              0
end_date             2842
campaign_id             0
is_deleted              0
class_name              0
price                   0
campaign_name           0
mean                    0
median                  0
max                     0
min                     0
routine_flg             0
calc_date               0
membership_period       0
exit_date            2842
period                  0
now_date                0
dtype: int64

In [48]:
predict_data = predict_data.dropna(subset=['count_1'])
predict_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3679 entries, 0 to 4070
Data columns (total 24 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   연월                 3679 non-null   object        
 1   customer_id        3679 non-null   object        
 2   count_0            3679 non-null   int64         
 3   count_1            3679 non-null   float64       
 4   name               3679 non-null   object        
 5   class              3679 non-null   object        
 6   gender             3679 non-null   object        
 7   start_date         3679 non-null   datetime64[ns]
 8   end_date           1052 non-null   object        
 9   campaign_id        3679 non-null   object        
 10  is_deleted         3679 non-null   float64       
 11  class_name         3679 non-null   object        
 12  price              3679 non-null   float64       
 13  campaign_name      3679 non-null   object        
 14  mean    

In [49]:
target_col = ['campaign_name', 'class_name', 'gender',  'count_1', 'routine_flg', 'period', 'is_deleted']
predict_data = predict_data[target_col]

In [50]:
predict_data.head(2)

,campaign_name,class_name,gender,count_1,routine_flg,period,is_deleted
0,일반,종일,F,4.0,1.0,20,0.0
1,일반,주간,F,2.0,1.0,30,0.0


In [51]:
# One-hot encoding
predict_data = pd.get_dummies(predict_data)
predict_data.head(2)

,count_1,routine_flg,period,is_deleted,campaign_name_일반,campaign_name_입회비무료,campaign_name_입회비반액할인,class_name_야간,class_name_종일,class_name_주간,gender_F,gender_M
0,4.0,1.0,20,0.0,1,0,0,0,1,0,1,0
1,2.0,1.0,30,0.0,1,0,0,0,0,1,1,0


In [52]:
del predict_data['gender_M']

In [53]:
# 의사 결정 트리
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split

In [54]:
# 데이터 분리, 데이터 비율 맞추기(큰 쪽을 줄이거나 작은 쪽을 늘리거나)
exit = predict_data.loc[predict_data['is_deleted']==1]
conti = predict_data.loc[predict_data['is_deleted']==0].sample(len(exit))

In [55]:
exit.shape

(1052, 11)

In [56]:
conti.shape

(1052, 11)

In [57]:
X = pd.concat([exit, conti], ignore_index=True)
y = X['is_deleted']

In [58]:
del X['is_deleted']

In [59]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=156)

In [60]:
from sklearn.metrics import accuracy_score

model = DecisionTreeClassifier(random_state=0, max_depth=5)
model.fit(X_train, y_train)
pred = model.predict(X_test)
accuracy_score(y_test, pred)

0.8878326996197718